Check:
* clean text vs original
* word2vec vs glove vs fasttext
* oversampling vs undersampling
* spellchecking vs no spellchecking

In [1]:
from sklearn.metrics import roc_auc_score
import logging
import pickle
from sklearn import metrics
from sklearn.metrics import f1_score
import keras
from keras.models import Model, Sequential
from keras.layers import *
from keras.preprocessing.text import one_hot
from keras.optimizers import Adam
from tqdm import tnrange, tqdm_notebook
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.utils import class_weight
from keras.callbacks import *
import gensim
import tensorflow as tf
import pandas as pd
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
set_session(tf.Session(config=config))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn import preprocessing

In [4]:
MODEL_PATH = "../models/"

In [5]:
DATA_PATH = "../data/"
data = pd.read_csv(DATA_PATH + "preprocessed/train_heavy_clean_no-stopwords.csv")

In [6]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [7]:
data.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,heavy_clean
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation edits made username hardcore metal...
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,' aww ! matches background colour seemingly st...
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"hey man , really trying edit war . guy constan..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,""" cannotmake real suggestions improvement - wo..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,", sir , hero . chance remember page ?"


In [8]:
X_train, X_test, y_train, y_test = train_test_split(data[['comment_text', 'heavy_clean']], data[list_classes], test_size=0.33, random_state=42)

In [9]:
tokenizer = Tokenizer(num_words=None, oov_token=0)
tokenizer.fit_on_texts(X_train['comment_text'])
word_index = tokenizer.word_index

In [59]:
# X_train.loc[pd.isnull(X_train['heavy_clean'])]['heavy_clean'] = X_train[pd.isnull(X_train['heavy_clean'])]['comment_text'].apply(heavy_clean)

# X_train.loc[8846]['heavy_clean'] = heavy_clean(X_train.loc[8846]['comment_text'])

# X_train[pd.isnull(X_train['heavy_clean'])]

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
embeddings_index_w2v = gensim.models.KeyedVectors.load_word2vec_format('../../../embeddings/GoogleNews-vectors-negative300.bin', binary = True)

In [11]:
max_length = 200

In [12]:
print('Found %s word vectors.' % len(embeddings_index_w2v.vocab))
embedding_matrix = np.zeros((len(word_index) + 2, 300))
oov = []
embedding_matrix[0] = np.random.rand(1, 300)
embedding_matrix[-1] = np.zeros((1, 300))
for word, i in word_index.items():
    word = str(word)
    if word in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.lower()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.capitalize() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.capitalize()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower().capitalize() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.lower().capitalize()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    else:
        oov.append(word)
embedding_size = 300

Found 3000000 word vectors.


In [13]:
len(word_index)

164436

In [14]:
len(set(word_index) & set(embeddings_index_w2v.vocab))

58515

In [15]:
len(oov)

80765

In [16]:
embedding_matrix

array([[ 0.81057241,  0.26098693,  0.14808145, ...,  0.86761916,
         0.30566176,  0.29109141],
       [ 0.07468531,  0.09791063,  0.04645062, ...,  0.00341549,
         0.04440133, -0.06421115],
       [ 0.01803273,  0.0796839 ,  0.03568782, ...,  0.03077838,
        -0.01652214, -0.09403456],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04803049, -0.03817808,  0.03217427, ...,  0.00434891,
        -0.0180884 ,  0.08744012],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [17]:
embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_size,
                            weights=[embedding_matrix],
                            input_length=max_length, trainable = False)

In [72]:
tokenizer_clean = Tokenizer(num_words=None, oov_token=0)
tokenizer_clean.fit_on_texts(X_train['heavy_clean'])
word_index_clean = tokenizer_clean.word_index

In [73]:
print('Found %s word vectors.' % len(embeddings_index_w2v.vocab))
embedding_matrix_clean = np.zeros((len(word_index_clean) + 2, 300))
oov_clean = []
embedding_matrix_clean[0] = np.random.rand(1, 300)
embedding_matrix_clean[-1] = np.zeros((1, 300))
for word, i in word_index_clean.items():
    word = str(word)
    if word in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word]
        embedding_matrix_clean[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.lower()]
        embedding_matrix_clean[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.capitalize() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.capitalize()]
        embedding_matrix_clean[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower().capitalize() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.lower().capitalize()]
        embedding_matrix_clean[i] = embedding_vector / np.linalg.norm(embedding_vector)
    else:
        oov_clean.append(word)
embedding_size = 300
embedding_layer_clean = Embedding(embedding_matrix_clean.shape[0],
                            embedding_size,
                            weights=[embedding_matrix_clean],
                            input_length=max_length, trainable = False)

Found 3000000 word vectors.


In [18]:
adam = keras.optimizers.Adam(clipnorm=1.)

In [16]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

# Clean Text vs Original

In [21]:
x_train_texts = tokenizer.texts_to_sequences(X_train['comment_text'])
x_train_texts = pad_sequences(x_train_texts, maxlen=max_length, padding='post')

x_test_texts = tokenizer.texts_to_sequences(X_test['comment_text'])
x_test_texts = pad_sequences(x_test_texts, maxlen=max_length, padding='post')

In [23]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
lstm1 = CuDNNLSTM(20, return_sequences=True)(embedded_sequences)
lstm2 = CuDNNLSTM(5)(lstm1)
output = Dense(units=6, activation='sigmoid')(lstm2)
model =  Model(sequence_input, output)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
model.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
model.fit(x_train_texts, y_train, validation_data=(x_test_texts, y_test),
              epochs=5, batch_size=5, callbacks=[RocAuc])

Train on 106912 samples, validate on 52659 samples
Epoch 1/5
106912/106912 [==============================] - 529s 5ms/step - loss: 0.0955 - acc: 0.9717 - val_loss: 0.0530 - val_acc: 0.9804

 ROC-AUC - epoch: 1 - score: 0.967719 

Epoch 2/5
106912/106912 [==============================] - 532s 5ms/step - loss: 0.0512 - acc: 0.9814 - val_loss: 0.0520 - val_acc: 0.9811

 ROC-AUC - epoch: 2 - score: 0.971533 

Epoch 3/5
106912/106912 [==============================] - 589s 6ms/step - loss: 0.0489 - acc: 0.9819 - val_loss: 0.0492 - val_acc: 0.9818

 ROC-AUC - epoch: 3 - score: 0.973195 

Epoch 4/5
106912/106912 [==============================] - 582s 5ms/step - loss: 0.0475 - acc: 0.9823 - val_loss: 0.0495 - val_acc: 0.9817

 ROC-AUC - epoch: 4 - score: 0.973869 

Epoch 5/5
106912/106912 [==============================] - 564s 5ms/step - loss: 0.0462 - acc: 0.9827 - val_loss: 0.0493 - val_acc: 0.9819

 ROC-AUC - epoch: 5 - score: 0.974547 



In [84]:
x_train_texts_clean = tokenizer_clean.texts_to_sequences(X_train['heavy_clean'])
x_train_texts_clean = pad_sequences(x_train_texts_clean, maxlen=max_length, padding='post')

x_test_texts_clean = tokenizer_clean.texts_to_sequences(X_test['heavy_clean'])
x_test_texts_clean = pad_sequences(x_test_texts_clean, maxlen=max_length, padding='post')

In [86]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer_clean(sequence_input)
lstm1 = CuDNNLSTM(20, return_sequences=True)(embedded_sequences)
lstm2 = CuDNNLSTM(5)(lstm1)
output = Dense(units=6, activation='sigmoid')(lstm2)
model_clean =  Model(sequence_input, output)
model_clean.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
model_clean.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
model_clean.fit(x_train_texts_clean, y_train, validation_data=(x_test_texts_clean, y_test),
              epochs=5, batch_size=5, callbacks=[RocAuc])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 300)          45433500  
_________________________________________________________________
cu_dnnlstm_5 (CuDNNLSTM)     (None, 200, 20)           25760     
_________________________________________________________________
cu_dnnlstm_6 (CuDNNLSTM)     (None, 5)                 540       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 36        
Total params: 45,459,836
Trainable params: 26,336
Non-trainable params: 45,433,500
_________________________________________________________________
Train on 106912 samples, validate on 52659 samples
Epoch 1/5
106912/106912 [==============================] - 567s 5ms/step -

## Char vs Word

In [87]:
tokenizer_char = Tokenizer(num_words=None, oov_token=0, char_level=True)
tokenizer_char.fit_on_texts(X_train['comment_text'])
char_index = tokenizer_char.word_index

In [88]:
embeddings_index = {}
f = open("../../../embeddings/pretrained_character_embeddings.txt")
for line in f:
    if line[0] != ' ':
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
    else:
        values = line.split()
        word = ' '
        coefs = np.asarray(values, dtype='float32')
    embeddings_index[word] = coefs
f.close()
embeddings_index['\n'] = embeddings_index.pop('\\n')

In [89]:
print('Found %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((len(char_index) + 1, 300))
oov = []
for char, i in char_index.items():
    try:
        embedding_vector = embeddings_index[char]
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    except Exception as e:
        oov.append(char)
embedding_size = 300

Found 91 word vectors.


In [92]:

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_size,
                            weights=[embedding_matrix],
                            input_length=max_length, trainable = False, mask_zero = False)

In [90]:
x_train_texts = tokenizer_char.texts_to_sequences(X_train['comment_text'])
x_train_texts = pad_sequences(x_train_texts, maxlen=max_length, padding='post')

x_test_texts = tokenizer_char.texts_to_sequences(X_test['comment_text'])
x_test_texts = pad_sequences(x_test_texts, maxlen=max_length, padding='post')

In [94]:
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
lstm1 = CuDNNLSTM(20, return_sequences=True)(embedded_sequences)
lstm2 = CuDNNLSTM(5)(lstm1)
output = Dense(units=6, activation='sigmoid')(lstm2)
model_char =  Model(sequence_input, output)
model_char.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
model_char.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
model_char.fit(x_train_texts, y_train, validation_data=(x_test_texts, y_test),
              epochs=5, batch_size=5, callbacks=[RocAuc])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 200, 300)          612000    
_________________________________________________________________
cu_dnnlstm_10 (CuDNNLSTM)    (None, 200, 20)           25760     
_________________________________________________________________
cu_dnnlstm_11 (CuDNNLSTM)    (None, 5)                 540       
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 36        
Total params: 638,336
Trainable params: 26,336
Non-trainable params: 612,000
_________________________________________________________________
Train on 106912 samples, validate on 52659 samples
Epoch 1/5
106912/106912 [==============================] - 574s 5ms/step - loss:

## Glove vs FastText vs word2vec

In [11]:
max_length = 200

x_train_texts = tokenizer.texts_to_sequences(X_train['comment_text'])
x_train_texts = pad_sequences(x_train_texts, maxlen=max_length, padding='post')

x_test_texts = tokenizer.texts_to_sequences(X_test['comment_text'])
x_test_texts = pad_sequences(x_test_texts, maxlen=max_length, padding='post')

In [12]:
embeddings_index_w2v = gensim.models.KeyedVectors.load_word2vec_format('../../../embeddings/GoogleNews-vectors-negative300.bin', binary = True)

In [15]:
print('Found %s word vectors.' % len(embeddings_index_w2v.vocab))
embedding_matrix = np.zeros((len(word_index) + 2, 300))
oov = []
embedding_matrix[0] = np.random.rand(1, 300)
embedding_matrix[-1] = np.zeros((1, 300))
for word, i in word_index.items():
    word = str(word)
    if word in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.lower()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.capitalize() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.capitalize()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower().capitalize() in embeddings_index_w2v.vocab:
        embedding_vector = embeddings_index_w2v[word.lower().capitalize()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    else:
        oov.append(word)
embedding_size = 300
embedding_layer_w2v = Embedding(embedding_matrix.shape[0],
                            embedding_size,
                            weights=[embedding_matrix],
                            input_length=max_length, trainable = False)
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer_w2v(sequence_input)
lstm1 = CuDNNLSTM(20, return_sequences=True)(embedded_sequences)
lstm2 = CuDNNLSTM(5)(lstm1)
output = Dense(units=6, activation='sigmoid')(lstm2)
modelw2v =  Model(sequence_input, output)
modelw2v.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
modelw2v.summary()

Found 3000000 word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          49331400  
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 200, 20)           25760     
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 5)                 540       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 36        
Total params: 49,357,736
Trainable params: 26,336
Non-trainable params: 49,331,400
_________________________________________________________________


NameError: name 'RocAucEvaluation' is not defined

In [17]:

RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
modelw2v.fit(x_train_texts, y_train, validation_data=(x_test_texts, y_test),
              epochs=5, batch_size=5, callbacks=[RocAuc])

Train on 106912 samples, validate on 52659 samples
Epoch 1/5
106912/106912 [==============================] - 601s 6ms/step - loss: 0.1448 - acc: 0.9631 - val_loss: 0.1412 - val_acc: 0.9630

 ROC-AUC - epoch: 1 - score: 0.515201 

Epoch 2/5
106912/106912 [==============================] - 593s 6ms/step - loss: 0.0775 - acc: 0.9735 - val_loss: 0.0553 - val_acc: 0.9809

 ROC-AUC - epoch: 2 - score: 0.970679 

Epoch 3/5
106912/106912 [==============================] - 583s 5ms/step - loss: 0.0522 - acc: 0.9813 - val_loss: 0.0521 - val_acc: 0.9808

 ROC-AUC - epoch: 3 - score: 0.972156 

Epoch 4/5
106912/106912 [==============================] - 739s 7ms/step - loss: 0.0491 - acc: 0.9821 - val_loss: 0.0507 - val_acc: 0.9811

 ROC-AUC - epoch: 4 - score: 0.973756 

Epoch 5/5
106912/106912 [==============================] - 964s 9ms/step - loss: 0.0475 - acc: 0.9825 - val_loss: 0.0525 - val_acc: 0.9815

 ROC-AUC - epoch: 5 - score: 0.973140 



In [18]:
from gensim.models.wrappers import FastText
embeddings_index_fasttext = gensim.models.KeyedVectors.load_word2vec_format('../../../embeddings/crawl-300d-2M.vec')

In [19]:
print('Found %s word vectors.' % len(embeddings_index_fasttext.vocab))
embedding_matrix = np.zeros((len(word_index) + 2, 300))
oov = []
embedding_matrix[0] = np.random.rand(1, 300)
embedding_matrix[-1] = np.zeros((1, 300))
for word, i in word_index.items():
    word = str(word)
    if word in embeddings_index_fasttext.vocab:
        embedding_vector = embeddings_index_fasttext[word]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower() in embeddings_index_fasttext.vocab:
        embedding_vector = embeddings_index_fasttext[word.lower()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.capitalize() in embeddings_index_fasttext.vocab:
        embedding_vector = embeddings_index_fasttext[word.capitalize()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower().capitalize() in embeddings_index_fasttext.vocab:
        embedding_vector = embeddings_index_fasttext[word.lower().capitalize()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    else:
        oov.append(word)
embedding_size = 300
embedding_layer_fasttext = Embedding(embedding_matrix.shape[0],
                            embedding_size,
                            weights=[embedding_matrix],
                            input_length=max_length, trainable = False)
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer_fasttext(sequence_input)
lstm1 = CuDNNLSTM(20, return_sequences=True)(embedded_sequences)
lstm2 = CuDNNLSTM(5)(lstm1)
output = Dense(units=6, activation='sigmoid')(lstm2)
modelfasttext =  Model(sequence_input, output)
modelfasttext.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
modelfasttext.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
modelfasttext.fit(x_train_texts, y_train, validation_data=(x_test_texts, y_test),
              epochs=5, batch_size=5, callbacks=[RocAuc])

Found 2000000 word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 300)          49331400  
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 200, 20)           25760     
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 5)                 540       
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 36        
Total params: 49,357,736
Trainable params: 26,336
Non-trainable params: 49,331,400
_________________________________________________________________
Train on 106912 samples, validate on 52659 samples
Epoch 1/5
106912/106912 [=====================

In [20]:
GLOVE_FILE = '../../../embeddings/glove.42B.300d.txt'
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(GLOVE_FILE))

In [ ]:
print('Found %s word vectors.' % len(embeddings_index.vocab))
embedding_matrix = np.zeros((len(word_index) + 2, 300))
oov = []
embedding_matrix[0] = np.random.rand(1, 300)
embedding_matrix[-1] = np.zeros((1, 300))
for word, i in word_index.items():
    word = str(word)
    if word in embeddings_index.vocab:
        embedding_vector = embeddings_index_glove[word]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower() in embeddings_index_glove.vocab:
        embedding_vector = embeddings_index_glove[word.lower()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.capitalize() in embeddings_index_glove.vocab:
        embedding_vector = embeddings_index_glove[word.capitalize()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    elif word.lower().capitalize() in embeddings_index_glove.vocab:
        embedding_vector = embeddings_index_glove[word.lower().capitalize()]
        embedding_matrix[i] = embedding_vector / np.linalg.norm(embedding_vector)
    else:
        oov.append(word)
embedding_size = 300
embedding_layer_glove = Embedding(embedding_matrix.shape[0],
                            embedding_size,
                            weights=[embedding_matrix],
                            input_length=max_length, trainable = False)
sequence_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer_glove(sequence_input)
lstm1 = CuDNNLSTM(20, return_sequences=True)(embedded_sequences)
lstm2 = CuDNNLSTM(5)(lstm1)
output = Dense(units=6, activation='sigmoid')(lstm2)
modelglove =  Model(sequence_input, output)
modelglove.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])    
modelglove.summary()
RocAuc = RocAucEvaluation(validation_data=(x_test_texts, y_test), interval=1)
modelglove.fit(x_train_textsglove, y_train, validation_data=(x_test_textsglove, y_test),
              epochs=5, batch_size=5, callbacks=[RocAuc])